In [ ]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf

# Load the saved model and scaler
model = tf.keras.models.load_model('ann_model.h5')
scaler = joblib.load('scaler.pkl')

# Get the input file name from the user
input_file = input("Enter the name of the input CSV file (with .csv extension): ")

# Read the input CSV file
try:
    data = pd.read_csv(input_file)
except FileNotFoundError:
    print(f"File {input_file} not found. Please check the file path and try again.")
    exit()

# Check if required columns exist
if not all(col in data.columns for col in ['M', 'h', 'd']):
    print("The input CSV file must contain 'M' (local magnitude), 'h' (depth), and 'd' (distance) columns.")
    exit()

# Create a new dataframe to store results
output_data = data.copy()

# Calculate the hypocentral distance (R) and predict the PGA
R_values = []
PGA_values = []

for index, row in data.iterrows():
    M = row['M']
    h = row['h']
    d = row['d']
    
    # Calculate hypocentral distance R
    R = np.sqrt(h**2 + d**2)
    
    # Prepare the input features (M, R)
    X_new = np.array([[M, R]])  # Format as 2D array (1 sample, 2 features)
    
    # Normalize the input using the saved scaler
    X_new_scaled = scaler.transform(X_new)
    
    # Predict log10(PGA) using the trained model
    log_pga_pred = model.predict(X_new_scaled)
    
    # Convert log10(PGA) back to PGA (cm/s²)
    pga_pred = 10 ** log_pga_pred.flatten()
    
    # Append results
    R_values.append(R)
    PGA_values.append(pga_pred[0])

# Add the calculated R and predicted PGA to the output dataframe
output_data['R'] = R_values
output_data['PGA'] = PGA_values

# Generate the output file name
output_file = input_file.replace('.csv', '_pga.csv')

# Save the results to a new CSV file
output_data.to_csv(output_file, index=False)

print(f"Results have been saved to {output_file}")
